# imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
from nltk import download
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from nltk.stem import  WordNetLemmatizer

In [ ]:
download('stopwords')
download('wordnet')
download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Amazon Reviews/AMAZON_FASHION_v4.parquet')

In [ ]:
new = pd.concat([df[df.sentiment == 0].head(10000), df[df.sentiment == 1].head(10000)], ignore_index=True)
del df

In [ ]:
new.reviewLength.quantile(0.8)

50.0

# Modeling

In [ ]:
metrics = pd.DataFrame()

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Amazon Reviews/AMAZON_FASHION_v5.parquet')
df.shape

(20000, 51)

In [ ]:
df.columns

Index(['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10',
       'pc11', 'pc12', 'pc13', 'pc14', 'pc15', 'pc16', 'pc17', 'pc18', 'pc19',
       'pc20', 'pc21', 'pc22', 'pc23', 'pc24', 'pc25', 'pc26', 'pc27', 'pc28',
       'pc29', 'pc30', 'pc31', 'pc32', 'pc33', 'pc34', 'pc35', 'pc36', 'pc37',
       'pc38', 'pc39', 'pc40', 'pc41', 'pc42', 'pc43', 'pc44', 'pc45', 'pc46',
       'pc47', 'pc48', 'pc49', 'pc50', 'sentiment'],
      dtype='object')

In [ ]:
X = df.drop(columns=['sentiment'])
y = df.sentiment
del df

In [ ]:
X.shape, y.shape

((20000, 50), (20000,))

In [ ]:
def evaluate(yhat, y, model):
    global metrics
    metrics = pd.concat([metrics, pd.DataFrame(
        {
           'model': [model, ],
           'accuracy_score': [accuracy_score(y_pred=yhat, y_true=y),],
           'f1_score': [f1_score(y_pred=yhat, y_true=y),],
           'precision_score': [precision_score(y_pred=yhat, y_true=y),],
           'recall_score': [recall_score(y_pred=yhat, y_true=y),],
        }
    )]
              )

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay

In [ ]:
gnb = GaussianNB()
gnb.fit(X, y)

GaussianNB()

In [ ]:
yhat = gnb.predict(X)
evaluate(yhat, y, model='GNB')

## RandomForest

In [ ]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
rf = RandomForestClassifier()
grid = RandomizedSearchCV(
    rf,
    param_distributions={
        'bootstrap': [True, False],
        'criterion': ['gini', 'entropy'],
        'max_depth': [2,3,5],
        'max_features': ['sqrt', 'log'],
        'max_leaf_nodes': [10, 15],
        'n_estimators': [100, 50, 150],
        'random_state': [42,],
        },
    cv=5,
    scoring='accuracy'
)

In [ ]:
grid.fit(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 3, 5],
                                        'max_features': ['sqrt', 'log'],
                                        'max_leaf_nodes': [10, 15],
                                        'n_estimators': [100, 50, 150],
                                        'random_state': [42]},
                   scoring='accuracy')

In [ ]:
rf_best = grid.best_estimator_

In [ ]:
yhat = rf_best.predict(X)
evaluate(yhat, y, model='RF')

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform


In [ ]:
lr = LogisticRegression()

In [ ]:
# Define the hyperparameter search space
param_dist = {
    'penalty': ['l1', 'l2'],
    'C': uniform(0.1, 10),
    'solver': ['liblinear', 'saga']
}

In [ ]:
# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    lr, param_distributions=param_dist, n_iter=10, cv=5, random_state=42
)

# Fit the model to the entire dataset (without explicit train-test split)
random_search.fit(X, y)



RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x790d832a9ea0>,
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['liblinear', 'saga']},
                   random_state=42)

In [ ]:
lr_best = random_search.best_estimator_

In [ ]:
yhat = lr_best.predict(X)
evaluate(yhat, y, model='LR')

SVM- Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from scipy.stats import uniform, randint

In [ ]:
SVC().get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
svm = SVC()

In [ ]:
svm = SVC(kernel='linear', C=1.0)
svm.fit(X, y)

SVC(kernel='linear')

In [ ]:
yhat = svm.predict(X)
evaluate(yhat, y, model='SVM')

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb.fit(X,y)

GradientBoostingClassifier(random_state=42)

In [ ]:
yhat = gb.predict(X)
evaluate(yhat, y, model='GB')

# Extreme Gradient Boosting- XGB

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train (fit) the model on the training data
xgb.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
yhat = xgb.predict(X)
evaluate(yhat, y, model='XGB')

In [ ]:
metrics

,model,accuracy_score,f1_score,precision_score,recall_score
0,GNB,0.75260,0.772652,0.714723,0.8408
0,RF,0.77295,0.779595,0.757427,0.8031
0,LR,0.81765,0.824891,0.793387,0.8590
0,SVM,0.81610,0.826526,0.782182,0.8762
0,GB,0.83610,0.840626,0.818036,0.8645
0,XGB,0.82940,0.834994,0.808485,0.8633


In [ ]:
metrics.to_csv('/content/drive/MyDrive/Colab Notebooks/Amazon Reviews//metrics.csv')